In [1]:
import pandas as pd
from geopandas import GeoDataFrame, read_file

import sys
sys.path.append("..")
import movingpandas as mpd
mpd.show_versions()

import warnings
warnings.simplefilter("ignore")

/home/arg/miniconda3/envs/movingpandas/lib/python3.9/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)



MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.9.19 | packaged by conda-forge | (main, Mar 20 2024, 12:50:21)  [GCC 12.3.0]
executable : /home/arg/miniconda3/envs/movingpandas/bin/python
machine    : Linux-5.15.0-105-generic-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.8.4
GDAL data dir: /home/arg/miniconda3/envs/movingpandas/share/gdal
PROJ       : 9.3.1
PROJ data dir: /home/arg/miniconda3/envs/movingpandas/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.3
pandas     : 2.2.1
fiona      : 1.9.6
numpy      : 1.26.4
shapely    : 2.0.3
rtree      : 1.2.0
pyproj     : 3.6.1
matplotlib : 3.8.3
mapclassify: 2.6.1
geopy      : 2.4.1
holoviews  : 1.18.3
hvplot     : 0.9.2
geoviews   : 1.11.1
stonesoup  : 1.2


In [28]:
import csv
import os
import shutil
import xml.etree.ElementTree as ET
from datetime import timedelta

import numpy as np
from dateutil import parser


def generate_grid_and_path(x_min, x_max, x_n, y_min, y_max, y_n):
    x_values = np.linspace(x_min, x_max, x_n)
    y_values = np.linspace(y_min, y_max, y_n)
    X, Y = np.meshgrid(x_values, y_values)
    path = []

    for i in range(x_n):
        for j in range(y_n):
            if i % 2 == 0:
                path.append((X[j, i], Y[j, i]))
            else:
                path.append((X[-j - 1, i], Y[-j - 1, i]))

    if x_n % 2 == 0:
        for j in range(y_n):
            for i in range(x_n):
                if j % 2 == 0:
                    path.append((X[j, i], Y[j, i]))
                else:
                    path.append((X[j, -i - 1], Y[j, -i - 1]))
    else:
        for j in range(y_n - 1, -1, -1):
            for i in range(x_n):
                if j % 2 == 0:
                    path.append((X[j, i], Y[j, i]))
                else:
                    path.append((X[j, -i - 1], Y[j, -i - 1]))
    return path

'''
def parse_corrdinates_str_list(kml_path):
    tree = ET.parse(kml_path)
    root = tree.getroot()
    namespace = {"kml": "http://www.opengis.net/kml/2.2"}
    specific_name = "vehicle_global_position:14"
    #namespace = {"kml"}
    #specific_name = "R8039938565"
    coordinates = None
    coordinates_str_list = []
    for placemark in root.findall(".//kml:Placemark", namespaces=namespace):
        name = placemark.find("kml:name", namespaces=namespace)
        if name is not None and name.text == specific_name:
            # Find the LineString element
            line_string = placemark.find(".//kml:LineString", namespaces=namespace)
            if line_string is not None:
                # Extract the coordinates
                coordinates = line_string.find("kml:coordinates", namespaces=namespace)
                if coordinates is not None:
                    coordinates_str_list.append(coordinates.text.strip())
    return coordinates_str_list
'''
def parse_corrdinates_str_list(kml_path):
    tree = ET.parse(kml_path)
    root = tree.getroot()
    coordinates = None
    coordinates_str_list = []
    for placemark in root.findall(".//Placemark"):
        name = placemark.find("name")
        if name is not None :
            # Find the LineString element
            line_string = placemark.find(".//LineString")
            if line_string is not None:
                # Extract the coordinates
                coordinates = line_string.find("coordinates")
                if coordinates is not None:
                    coordinates_str_list.append(coordinates.text.strip())
    return coordinates_str_list

def get_means(coordinates):
    mean_x = np.mean([coord[0] for coord in coordinates])
    mean_y = np.mean([coord[1] for coord in coordinates])
    return mean_x, mean_y


def write_to_csv(
    csv_file_path, coordinates, trajectory_id, tracker, base_timestamp, mean_x, mean_y, grid_size_half=10e-5, xy_n=10
):
    base_time = parser.parse(base_timestamp)
    with open(csv_file_path, "w", newline="") as csv_file:
        writer = csv.writer(csv_file, delimiter=";")

        headers = ["X", "Y", "fid", "id", "sequence", "trajectory_id", "tracker", "t"]
        writer.writerow(headers)
        for i, coord in enumerate(coordinates, start=1):
            x, y, _ = coord
            timestamp = (base_time + timedelta(seconds=1 * (i - 1))).isoformat()
            row = [x, y, i, i, i, trajectory_id, tracker, timestamp]
            writer.writerow(row)

        path = generate_grid_and_path(
            mean_x - grid_size_half,
            mean_x + grid_size_half,
            xy_n,
            mean_y - grid_size_half,
            mean_y + grid_size_half,
            xy_n,
        )
        for i, coord in enumerate(path, start=len(coordinates) + 1):
            x, y = coord
            timestamp = (base_time + timedelta(seconds=1 * (i - 1))).isoformat()
            row = [x, y, i, i, i, trajectory_id + 1, tracker, timestamp]
            writer.writerow(row)

    print(f"Saved to {csv_file_path}")


def convert_kml_to_csv(kml_path):
    kml_name = kml_path.split("/")[-1].split(".")[0]

    #print(parse_corrdinates_str_list(kml_path))
    coordinates_str_list = parse_corrdinates_str_list(kml_path)[0].split()
    coordinates = [[float(coord) for coord in coord_str.split(",")] for coord_str in coordinates_str_list]
    mean_x, mean_y = get_means(coordinates)
    print(f"Min x: {min([coord[0] for coord in coordinates])}, Min y: {min([coord[1] for coord in coordinates])}")
    print(f"Max x: {max([coord[0] for coord in coordinates])}, Max y: {max([coord[1] for coord in coordinates])}")
    xy_n =61
    grid_size_half = 1 / 111_111 * xy_n # Convert to meter

    print(f"Mean x: {mean_x}, Mean y: {mean_y}")

    trajectory_id = 1
    tracker = 19
    base_timestamp = "2008-12-11 04:42:14+00"

    csv_file_path = f"./data/{kml_name}.csv"
    
    write_to_csv(
        csv_file_path,
        coordinates,
        trajectory_id,
        tracker,
        base_timestamp,
        mean_x,
        mean_y,
        grid_size_half,
        xy_n=xy_n,
    )
    # shutil.copy(csv_file_path, os.path.expanduser("./data/"))
    print(f"Copied to ./data/{kml_name}.csv")


kml_pathes = [

    "飛行軌跡_20230716070634_R8039938565.kml",
    "飛行軌跡_20230716070703_R8059961112.kml",
    "飛行軌跡_20230716071226_R6234489650.kml",
    "飛行軌跡_20230716071856_R6311160153.kml",
    "飛行軌跡_20230716073025_R6965064834.kml"
]


kml_pathes = [f"./data/{kml_path}" for kml_path in kml_pathes]

for kml_path in kml_pathes:
    convert_kml_to_csv(kml_path)

Min x: 120.40715401554887, Min y: 23.397033617902608
Max x: 120.40757264714772, Max y: 23.39709086926291
Mean x: 120.40743116050774, Mean y: 23.39706630437371
Saved to ./data/飛行軌跡_20230716070634_R8039938565.csv
Copied to ./data/飛行軌跡_20230716070634_R8039938565.csv
Min x: 120.40708673139896, Min y: 23.397038591671322
Max x: 120.40817323236193, Max y: 23.397990704066423
Mean x: 120.40736054429281, Mean y: 23.397503256931202
Saved to ./data/飛行軌跡_20230716070703_R8059961112.csv
Copied to ./data/飛行軌跡_20230716070703_R8059961112.csv
Min x: 120.40815800528571, Min y: 23.397019111458047
Max x: 120.408623201147, Max y: 23.398005730878634
Mean x: 120.4083790425234, Mean y: 23.39743410975168
Saved to ./data/飛行軌跡_20230716071226_R6234489650.csv
Copied to ./data/飛行軌跡_20230716071226_R6234489650.csv
Min x: 120.40940571259064, Min y: 23.39705641075589
Max x: 120.40990199179414, Max y: 23.398038921162822
Mean x: 120.40967385199457, Mean y: 23.397493299638597
Saved to ./data/飛行軌跡_20230716071856_R6311160153.

In [29]:
import geoviews as gv
bing_maps_tile_source = gv.tile_sources.WMTS("http://ecn.t3.tiles.virtualearth.net/tiles/a{q}.jpeg?g=1")
hvplot_defaults = {'tiles':bing_maps_tile_source, 'frame_height':700, 'frame_width':700, 'colorbar':True}

In [30]:
%%time 
df = pd.read_csv('data/飛行軌跡_20230716070634_R8039938565.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 662 ms, sys: 0 ns, total: 662 ms
Wall time: 660 ms


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

![alt text](img/1.png)

In [31]:
%%time 
df = pd.read_csv('data/飛行軌跡_20230716070703_R8059961112.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 767 ms, sys: 0 ns, total: 767 ms
Wall time: 765 ms


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

![alt text](img/2.png)

In [32]:
%%time 
df = pd.read_csv('data/飛行軌跡_20230716071226_R6234489650.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 733 ms, sys: 3.78 ms, total: 737 ms
Wall time: 736 ms


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

![alt text](img/3.png)

In [33]:
%%time 
df = pd.read_csv('data/飛行軌跡_20230716071856_R6311160153.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 714 ms, sys: 3.97 ms, total: 718 ms
Wall time: 715 ms


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

![alt text](img/4.png)

In [34]:
%%time 
df = pd.read_csv('data/飛行軌跡_20230716073025_R6965064834.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 803 ms, sys: 7.85 ms, total: 811 ms
Wall time: 813 ms


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

![alt text](img/5.png)